In [7]:
import pandas as pd
import re
import os

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.utils import pad_sequences
from collections import Counter
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers = 8)

nltk.download('stopwords')
nltk.download('wordnet')


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/argen7um/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/argen7um/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_text_and_score(pathes):
    text_score = []
    for path in pathes:
        file_names = os.listdir(path)
        for file_name in file_names:
            with open(f'{path}/{file_name}', 'r') as file:
                text = file.read()
                score = int(file_name.split('.')[0].split('_')[-1])
            text_score.append([text, score])
    return text_score

In [9]:
path = r'../data/aclImdb'
corpus = get_text_and_score([f'{path}/train/pos/', f'{path}/train/neg/', 
                             f'{path}/test/pos/', f'{path}/test/neg/'])

In [10]:
dataset = pd.DataFrame(corpus, columns=('data', 'target'))

texts_count = 50000
dataset = dataset.iloc[:texts_count]

In [11]:
def clean_regex(text):
    text = re.sub('\"{2,3}', '', text)
    text = re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)
    text = re.sub(r'[.,!?-]', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'[^\x00-\x7f]', r'', text)
    
    return text

In [12]:
stopwords = [word for word in set(stopwords.words('english')) if word not in ('no', 'not')]
rex_to_rem_stopwords = '|'.join(f'\s{word}\s' for word in stopwords)

In [13]:
max_lenth = 1_000

In [14]:
def clean_regex(text):
    text = re.sub('\"{2,3}', '', text)
    text = re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)
    text = re.sub(r'[.,!?-]', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'[^\x00-\x7f]', r'', text)
    
    return text

def tokenize_text(text):
    return word_tokenize(text)

def split_words_by_space(text):
    return text.split()

def make_lower(word_list):
    return list(map(lambda t: t.lower(), word_list))

def lemmatize(word_list):
    return list(map(lemmatizer.lemmatize, word_list))

def remove_stop_words1(word_list):
    text = join_word_list(word_list)
    text = re.sub(rex_to_rem_stopwords, ' ', text)
    return split_words_by_space(text)


def add_bos_tag(word_list):
    word_list.insert(0, '<bos>')
    return word_list

def limit_sequence(word_list, max_lenth):
    return word_list[:max_lenth]

def join_word_list(word_list):
    return ' '.join(word_list)

def preprocess_text(text):
    word_list = tokenize_text(text)
    lower_word_list = make_lower(word_list)
    lemmatized_word_list = make_lower(lower_word_list)
    word_list_with_bos = add_bos_tag(remove_stop_words1(lemmatized_word_list))
    preprocessed_text = join_word_list(limit_sequence(word_list_with_bos, 1000))
    
    return preprocessed_text

In [18]:
threshold_count = 15
count_words = Counter(nltk.flatten(list(dataset['data'].parallel_apply(lambda a: clean_regex(tokenize_text(make_lower(a)))))))

stopwords = set(stopwords) + set(word for word in count_words if count_words[word] <= threshold_count)

# stopwords = sorted(list(stopwords))
# vocabulary = [word for word in count_words if count_words[word] > threshold_count and word not in stopwords]

TypeError: expected string or bytes-like object

In [19]:
dataset['data'].parallel_apply(lambda a: clean_regex(tokenize_text(make_lower(a))))

TypeError: expected string or bytes-like object